In [ ]:
!pip install xgboost
!pip install tensorflow

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

# from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping

import joblib


In [2]:
import kagglehub

path = kagglehub.dataset_download("sohails07/delhi-weather-and-aqi-dataset-2025")

print("Path to dataset files:", path)

ModuleNotFoundError: No module named 'kagglehub'

In [ ]:
import os

path = "/root/.cache/kagglehub/datasets/sohails07/delhi-weather-and-aqi-dataset-2025/versions/1"

files = os.listdir(path)
print(files)


In [3]:
import pandas as pd

df = pd.read_csv(os.path.join(path, "delhi-weather-aqi-2025.csv"))

df.head()

NameError: name 'os' is not defined

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52560 entries, 0 to 52559
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date_ist        52560 non-null  object 
 1   time_ist        52560 non-null  object 
 2   location        52560 non-null  object 
 3   lat             52560 non-null  float64
 4   lon             52560 non-null  float64
 5   temp_c          52560 non-null  float64
 6   humidity        52560 non-null  int64  
 7   pressure_mb     52560 non-null  float64
 8   windspeed_kph   52560 non-null  float64
 9   condition_text  52560 non-null  object 
 10  description     52560 non-null  object 
 11  aqi_index       52560 non-null  int64  
 12  pm2_5           52560 non-null  float64
 13  pm10            52560 non-null  float64
 14  co              52560 non-null  int64  
 15  no2             52560 non-null  float64
dtypes: float64(8), int64(3), object(5)
memory usage: 6.4+ MB


In [ ]:
df.describe()

,lat,lon,temp_c,humidity,pressure_mb,windspeed_kph,aqi_index,pm2_5,pm10,co,no2
count,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000,52560.000000
mean,28.608850,77.174567,25.024406,62.285940,983.162635,6.565820,287.364212,89.542432,300.462700,863.844273,35.442757
std,0.058927,0.096519,7.618231,23.454666,6.688582,3.478066,327.979524,55.363812,412.782899,600.845140,28.978905
min,28.527300,77.049400,5.700000,5.000000,966.100000,0.000000,56.000000,6.000000,6.100000,146.000000,1.700000
25%,28.556200,77.100000,19.500000,44.000000,977.500000,4.000000,152.000000,51.100000,80.600000,464.000000,14.900000
50%,28.609300,77.160100,26.500000,65.000000,983.400000,6.000000,170.000000,76.400000,151.800000,664.000000,26.900000
75%,28.646900,77.261800,30.500000,82.000000,988.900000,8.700000,238.000000,110.400000,325.000000,1078.000000,47.100000
max,28.704100,77.316000,43.800000,100.000000,998.800000,27.100000,2742.000000,443.300000,3263.400000,7432.000000,220.100000


In [ ]:
set(df['location'])

{'Anand Vihar',
 'Connaught Place',
 'Dwarka',
 'IGI Airport',
 'Okhla Phase III',
 'Rohini'}

In [ ]:
df['datetime'] = pd.to_datetime(df['date_ist'] + ' ' + df['time_ist'], dayfirst=True)

df = df.sort_values('datetime')

df = df.drop(['date_ist','time_ist','description'], axis=1)

df_agg = df.groupby('datetime').mean(numeric_only=True).reset_index()

df_agg.head()


,datetime,lat,lon,temp_c,humidity,pressure_mb,windspeed_kph,aqi_index,pm2_5,pm10,co,no2
0,2025-01-01 00:00:00,28.60885,77.174567,8.050000,100.000000,993.483333,3.000000,197.0,189.75,192.80,1749.333333,65.45
1,2025-01-01 01:00:00,28.60885,77.174567,7.766667,99.833333,992.900000,3.233333,198.0,177.65,180.65,1593.666667,51.70
2,2025-01-01 02:00:00,28.60885,77.174567,7.366667,99.666667,992.450000,4.200000,199.0,166.00,168.55,1473.166667,39.80
3,2025-01-01 03:00:00,28.60885,77.174567,7.383333,99.333333,992.250000,5.500000,200.5,154.85,157.40,1397.166667,30.25
4,2025-01-01 04:00:00,28.60885,77.174567,7.200000,99.333333,992.016667,6.416667,201.0,145.55,148.10,1356.333333,22.60


In [ ]:

df_agg['hour'] = df_agg['datetime'].dt.hour
df_agg['day'] = df_agg['datetime'].dt.day
df_agg['month'] = df_agg['datetime'].dt.month
df_agg['dayofweek'] = df_agg['datetime'].dt.dayofweek

df_agg['hour_sin'] = np.sin(2 * np.pi * df_agg['hour']/24)
df_agg['hour_cos'] = np.cos(2 * np.pi * df_agg['hour']/24)

df_agg = df_agg.drop(['hour'], axis=1)

df_agg.head()


,datetime,lat,lon,temp_c,humidity,pressure_mb,windspeed_kph,aqi_index,pm2_5,pm10,...,pm25_lag_12,pm25_lag_24,pm25_lag_48,pm25_lag_72,pm25_lag_168,day,month,dayofweek,hour_sin,hour_cos
336,2025-01-15 00:00:00,28.60885,77.174567,11.050000,93.000000,993.500000,0.750000,182.5,1.887948,274.75,...,1.694698,1.672001,1.757630,1.706021,1.715723,15,1,2,0.000000,1.000000
337,2025-01-15 01:00:00,28.60885,77.174567,10.566667,94.833333,993.433333,0.400000,187.0,1.887167,274.30,...,1.672498,1.667323,1.754977,1.690104,1.701962,15,1,2,0.258819,0.965926
338,2025-01-15 02:00:00,28.60885,77.174567,9.666667,98.333333,993.033333,0.716667,191.0,1.885618,271.50,...,1.653246,1.667195,1.754615,1.675695,1.688542,15,1,2,0.500000,0.866025
339,2025-01-15 03:00:00,28.60885,77.174567,8.750000,99.000000,993.050000,0.750000,195.5,1.883735,268.20,...,1.641023,1.669869,1.753157,1.668728,1.675695,15,1,2,0.707107,0.707107
340,2025-01-15 04:00:00,28.60885,77.174567,8.783333,98.666667,993.216667,1.100000,200.0,1.887976,273.35,...,1.653806,1.673365,1.749299,1.671126,1.665126,15,1,2,0.866025,0.500000


In [ ]:

df_agg = df.groupby('datetime').mean(numeric_only=True).reset_index()

df_agg['pm2_5'] = np.log1p(df_agg['pm2_5'])

lags = [1, 3, 6, 12, 24, 48, 72, 168]
for lag in lags:
    df_agg[f'pm25_lag_{lag}'] = df_agg['pm2_5'].shift(lag)

df_agg = df_agg.dropna()


In [ ]:

target = 'pm2_5'

X = df_agg.drop(['datetime', target], axis=1)
y = df_agg[target]

temp_model = XGBRegressor(n_estimators=100)
temp_model.fit(X, y)

selector = SelectFromModel(temp_model, prefit=True, threshold='median')
X_selected = selector.transform(X)

selected_features = X.columns[selector.get_support()]
print("Selected Features:", selected_features)


Selected Features: Index(['humidity', 'pm10', 'co', 'no2', 'pm25_lag_1', 'pm25_lag_3',
       'pm25_lag_6', 'pm25_lag_12', 'pm25_lag_24'],
      dtype='object')


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


In [ ]:

feature_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()

X_scaled = feature_scaler.fit_transform(X[selected_features])
y_scaled = target_scaler.fit_transform(y.values.reshape(-1,1))


In [ ]:

def create_sequences(X, y, input_steps=48, output_steps=24):
    Xs, ys = [], []

    for i in range(len(X) - input_steps - output_steps):
        Xs.append(X[i:i+input_steps])
        ys.append(y[i+input_steps:i+input_steps+output_steps])

    return np.array(Xs), np.array(ys)

X_seq, y_seq = create_sequences(X_scaled, y_scaled, input_steps=168, output_steps=24)

print("X shape:", X_seq.shape)
print("y shape:", y_seq.shape)


X shape: (8400, 168, 9)
y shape: (8400, 24, 1)


In [ ]:

split = int(len(X_seq) * 0.8)

X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (6720, 168, 9)
Test shape: (1680, 168, 9)


In [ ]:

model_lstm = Sequential()

model_lstm.add(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(Dropout(0.2))

model_lstm.add(LSTM(64))
model_lstm.add(Dropout(0.2))

model_lstm.add(Dense(64, activation='relu'))
model_lstm.add(Dense(24))

model_lstm.compile(optimizer='adam', loss='mse')

model_lstm.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                   │ (None, 168, 128)       │        70,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 168, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_4 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 24)             │         1,560 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 125,784 (491.34 KB)

 Trainable params: 125,784 (491.34 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)

history = model_lstm.fit(
    X_train, y_train,
    validation_split=0.1,
    epochs=30,
    batch_size=32,
    callbacks=[early_stop]
)


Epoch 1/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - loss: 0.0793 - val_loss: 0.0088
Epoch 2/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0129 - val_loss: 0.0081
Epoch 3/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0108 - val_loss: 0.0066
Epoch 4/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0097 - val_loss: 0.0064
Epoch 5/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0087 - val_loss: 0.0061
Epoch 6/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0082 - val_loss: 0.0065
Epoch 7/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0082 - val_loss: 0.0062
Epoch 8/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0079 - val_loss: 0.0061
Epoch 9/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - loss: 0.0075 - val_loss: 0.0061
Epoch 10/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - loss: 0.0074 - val_loss: 0.0060
Epoch 11/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0074 - val_loss: 0.0060
Epoch 12/30
189/189 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:

lstm_train_pred = model_lstm.predict(X_train)
lstm_test_pred = model_lstm.predict(X_test)

lstm_train_pred_flat = lstm_train_pred.reshape(lstm_train_pred.shape[0], -1)
lstm_test_pred_flat = lstm_test_pred.reshape(lstm_test_pred.shape[0], -1)


210/210 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step


In [ ]:

X_train_last = X_train[:, -1, :]
X_test_last = X_test[:, -1, :]

X_train_fusion = np.concatenate([X_train_last, lstm_train_pred_flat], axis=1)
X_test_fusion = np.concatenate([X_test_last, lstm_test_pred_flat], axis=1)

y_train_flat = y_train.reshape(y_train.shape[0], -1)
y_test_flat = y_test.reshape(y_test.shape[0], -1)


In [ ]:

xgb_models = []

for i in range(24):
    model = XGBRegressor(n_estimators=200)
    model.fit(X_train_fusion, y_train_flat[:, i])
    xgb_models.append(model)

print("24 XGBoost models trained")


24 XGBoost models trained


In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

lstm_test_pred = model_lstm.predict(X_test)

lstm_test_pred_log = target_scaler.inverse_transform(
    lstm_test_pred.reshape(-1, 1)
).reshape(lstm_test_pred.shape)

y_test_log = target_scaler.inverse_transform(
    y_test.reshape(-1, 1)
).reshape(y_test.shape)

lstm_test_pred_actual = np.expm1(lstm_test_pred_log)
y_test_actual = np.expm1(y_test_log)

lstm_pred_flat = lstm_test_pred_actual.flatten()
y_test_flat = y_test_actual.flatten()

lstm_mae = mean_absolute_error(y_test_flat, lstm_pred_flat)
lstm_rmse = np.sqrt(mean_squared_error(y_test_flat, lstm_pred_flat))
lstm_r2 = r2_score(y_test_flat, lstm_pred_flat)

print("LSTM Performance (Original µg/m³ scale):")
print("MAE :", lstm_mae)
print("RMSE:", lstm_rmse)
print("R2  :", lstm_r2)

53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
LSTM Performance (Original µg/m³ scale):
MAE : 28.366289847001198
RMSE: 41.089284705068856
R2  : 0.5084414013709933


In [ ]:
# Save LSTM model
model_lstm.save("lstm_model.h5")

# Save XGBoost models
for i, model in enumerate(xgb_models):
    joblib.dump(model, f"xgb_model_step_{i}.pkl")

# Save scalers
joblib.dump(feature_scaler, "feature_scaler.pkl")
joblib.dump(target_scaler, "target_scaler.pkl")

print("All models saved successfully")
